# Breast Cancer Detection

## Preprocessing

* Number of instances: 569

* Number of attributes: 32

* Attribute information:
   
   1) ID number

   2) Diagnosis (M = malignant, B = benign)
3-32)


Ten real-valued features are computed for each cell nucleus:

  a) radius (mean of distances from center to points on the perimeter)

  b) texture (standard deviation of gray-scale values)

  c) perimeter

  d) area

  e) smoothness (local variation in radius lengths)

  f) compactness (perimeter^2 / area - 1.0)

  g) concavity (severity of concave portions of the contour)

  h) concave points (number of concave portions of the contour)

  i) symmetry

  j) fractal dimension ("coastline approximation" - 1)


* Missing attribute values: None

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#  Import and read the breast-cancer.data.csv.
df = pd.read_csv("../Resources/data.csv")
df.head()

The key challenge against its detection is how to classify tumors into malignant (cancerous) or benign(non-cancerous). We ask you to complete the analysis of classifying these tumors using machine learning (with SVMs) and the Breast Cancer Wisconsin (Diagnostic) Dataset.

In [ ]:
# Find null values
for column in df.columns:
    print(f"Column {column} has {df[column].isnull().sum()} null values")

In [ ]:
# Find duplicate entries
print(f"Duplicate entries: {df.duplicated().sum()}")

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
# Choose a cutoff value and create a list of diagnosis to be replaced
# use the variable name `diagnosis_to_replace`

# Transform diagnosis
def diagnosis_to_replace(diagnosis):
    if diagnosis == "M":
        return 1
    else:
        return 0
    

df["diagnosis"] = df["diagnosis"].apply(diagnosis_to_replace)
df.head(20)

In [ ]:
# Split our preprocessed data into our features and target arrays also drop the id as that is not useful
X = df.drop(["diagnosis", "id"], axis='columns')
y = df["diagnosis"]
X                   

# Explore the dataset

In [ ]:
# A quick look at the features to see if the data is distributed normally or not
X.hist(figsize=(15,15))
plt.show()

There is a significant number of features that have a strong right skew, so for some models the data will need to be transformed.

In [ ]:
benign = df.loc[df['diagnosis']==0]
benign.drop(columns=['id','diagnosis'], inplace=True)

In [ ]:
benign

In [ ]:
malign = df.loc[df['diagnosis']==1]
malign.drop(columns=['id','diagnosis'], inplace=True)

In [ ]:
malign

In [ ]:
# have a look at how the malign data looks compared to the benign in a few categories
plt.hist(benign['radius_mean'], alpha=.5, label='B')
plt.hist(malign['radius_mean'], alpha=.5, label='M')
plt.legend(loc='upper right')
plt.show()

In [ ]:
plt.hist(benign['texture_mean'], alpha=.5, label='B')
plt.hist(malign['texture_mean'], alpha=.5, label='M')
plt.legend(loc='upper right')
plt.show()

In [ ]:
plt.hist(benign['perimeter_mean'], alpha=.5, label='B')
plt.hist(malign['perimeter_mean'], alpha=.5, label='M')
plt.legend(loc='upper right')
plt.show()

In [ ]:
plt.hist(benign['smoothness_mean'], alpha=.5, label='B')
plt.hist(malign['smoothness_mean'], alpha=.5, label='M')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# We can come back to this when we want to tweak the features for improvements

In [ ]:
# check for any negative values in df
(df < 0).values.any()

In [ ]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a MinMaxScaler instance since all values are positive to try for better results
scaler = MinMaxScaler()

# Fit the MinMaxScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
#  Trial one: Logistic Regression Algorithm raw
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state = 78, max_iter=10000)
lr.fit(X_train, y_train)

print(f"Training Data Score: {lr.score(X_train, y_train)}")
print(f"Testing Data Score: {lr.score(X_test, y_test)}")

In [ ]:
#  Trial two: Random Forest Classifier raw
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=78, n_estimators=200).fit(X_train_scaled, y_train)

print(f'Training Score: {rfc.score(X_train_scaled, y_train)}')
print(f'Testing Score: {rfc.score(X_test_scaled, y_test)}')

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation ="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")